In [1]:
#importing dataset
import pandas as pd
df= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\train.csv")
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
#preprocessing 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
import string

df['new_tweet'] = df.tweet.str.replace('@user', '')
df['new_tweet'] = df['new_tweet'].str.replace("[^a-zA-Z#]", " " )
df['new_tweet'] = df['new_tweet'].str.replace("#", " ")
df['new_tweet'] = df['new_tweet'].apply(lambda x: x.split())
df['new_tweet']= df['new_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
def process(text):
    # Check characters to see if they are in punctuation
    nopunc = set(char for char in list(text) if char not in string.punctuation)
    # Join the characters to form the string.
    nopunc = " ".join(nopunc)
    # remove any stopwords if present
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords]
df['new_tweet'] = df['new_tweet'].apply(process)
df=df.drop(columns='tweet')
df=df.drop(columns='id')
df.head()


<ipython-input-2-d574088bc6ff>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['new_tweet'] = df['new_tweet'].str.replace("[^a-zA-Z#]", " " )


,label,new_tweet
0,0,"[drag, selfish, run, dysfunct, father, kid]"
1,0,"[thank, offer, getthank, use, wheelchair, lyft..."
2,0,"[majesti, bihday]"
3,0,"[ur, take, model, love, u, time]"
4,0,"[factsguid, motiv, societi]"


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test =train_test_split(df["new_tweet"], df["label"], test_size = 0.2, random_state = 42)
vectorizer = TfidfVectorizer(stop_words='english',norm='l2',sublinear_tf=True)
x_train_tfidf = vectorizer.fit_transform(x_train.apply(lambda x: ' '.join(x)))
x_test_tfidf = vectorizer.transform(x_test.apply(lambda x: ' '.join(x)))
model = RandomForestClassifier(n_estimators=200)
model.fit(x_train_tfidf,y_train)
predictions = model.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)

96.05818864382918


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test =train_test_split(df["new_tweet"], df["label"], test_size = 0.2, random_state = 42)
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(x_train.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
x_test_counts = count_vect.transform(x_test.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
model = RandomForestClassifier(n_estimators=200)
model.fit(x_train_tfidf,y_train)
predictions = model.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english',norm='l2',sublinear_tf=True)
x_train = vectorizer.fit_transform(df['new_tweet'].apply(lambda x: ' '.join(x)))

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
xcounts = count_vect.fit_transform(df['new_tweet'].apply(lambda x: ' '.join(x)))
x_train = transformer.fit_transform(xcounts)

In [12]:
import numpy as np
N = 10    
split=np.array_split(df, N)

In [13]:
x_tfidf=vectorizer.fit_transform(split[0]['new_tweet'].apply(lambda x:''.join(x)))

xcounts = count_vect.transform(split[0]['new_tweet'].apply(lambda x: ' '.join(x)))
x_train = transformer.transform(xcounts)

In [14]:
y_train=split[0]['label']

In [15]:
from modAL.models import ActiveLearner
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from modAL.uncertainty import entropy_sampling, uncertainty_sampling, margin_sampling
learner = ActiveLearner(estimator=RandomForestClassifier(),query_strategy=entropy_sampling,X_training=x_tfidf, y_training=y_train.ravel())


In [16]:
for i in range(1,10):
  x_t= vectorizer.transform(split[i]['new_tweet'].apply(lambda x: ' '.join(x)))
  accuracy_scores = [learner.score(x_t, split[i]['label'])]
  print(accuracy_scores)

[0.9289959336878324]
[0.932415519399249]
[0.9283479349186483]
[0.931476846057572]
[0.934918648310388]
[0.9249061326658323]
[0.925531914893617]
[0.9327284105131415]
[0.9286608260325406]


for i in range(1,10):
  x_c = count_vect.transform(split[i]['new_tweet'].apply(lambda x: ' '.join(x)))
  x_t= transformer.transform(x_c)
  accuracy_scores = [learner.score(x_t, split[i]['label'])]
  print(accuracy_scores)

In [17]:
df.loc[2915]

label                             0
new_tweet    [go, gettingther, day]
Name: 2915, dtype: object

In [18]:
for i in range(1,10):
    x_tfidf = vectorizer.transform(split[i]['new_tweet'].apply(lambda x: ' '.join(x)))
    query_idx, query_sample = learner.query(x_tfidf)
    split[i]=split[i].reset_index()
    split[i]=split[i].copy()
    split[i]=split[i].drop(['index'],axis=1)
    print(query_idx)
    print(split[i].loc[query_idx])
    #y=int(input("Enter the label"))
    learner.teach(x_tfidf[query_idx],[0])

[0]
   label                                          new_tweet
0      0  [gt, surpris, ka, make, read, betray, scifi, k...
[0]
   label                                       new_tweet
0      0  [react, life, happen, said, motivati, percent]
[0]
   label                                          new_tweet
0      0  [thank, got, night, result, text, twitter, boy...
[0]
   label                        new_tweet
0      0  [great, morn, day, good, night]
[0]
   label             new_tweet
0      0  [thank, rain, posit]
[0]
   label                                          new_tweet
0      0  [first, watch, showtim, episod, know, time, de...
[0]
   label                                new_tweet
0      0  [task, full, day, prepar, activ, first]
[0]
   label                                          new_tweet
0      1  [philosoph, meet, nationalist, white, aleksand...
[0]
   label                                        new_tweet
0      0  [stuff, cloth, readi, day, clear, new, holiday]


for i in range(1,10):
    x_counts = count_vect.transform(split[i]['new_tweet'].apply(lambda x: ' '.join(x)))
    x_tfidf = transformer.transform(x_counts)
    query_idx, query_sample = learner.query(x_tfidf)
    split[i]=split[i].reset_index()
    split[i]=split[i].copy()
    split[i]=split[i].drop(['index'],axis=1)
    print(split[i].loc[query_idx])
    y=int(input("Enter the label"))
    learner.teach(x_tfidf[query_idx],[y])

In [20]:
x_test_tfidf = vectorizer.transform(x_test.apply(lambda x: ' '.join(x)))
predictions = learner.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)

92.86719849835758
